In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold

# Raw Data Loading
train = pd.read_csv('./data/digit-recognizer/train.csv')
test = pd.read_csv('./data/digit-recognizer/test.csv')

# # Data Split
# # 7:3 비율로 train과 test 분리
# # x_data_test, t_data_test는 맨끝에서 모델의 최종 Accuracy를 측정할 때 딱 한번 사용
# x_data_train, x_data_test, t_data_train, t_data_test = \
# train_test_split(df[['height','weight']], df['label'], test_size=0.3, random_state=0)

# # Normalization
# scaler = MinMaxScaler()
# scaler.fit(x_data_train)  # scaling을 하기 위한 정보를 scaler에 setting

# x_data_train_norm = scaler.transform(x_data_train)
# x_data_test_norm = scaler.transform(x_data_test)

# # Tensorflow 구현
# # 해당 데이터는 Multinomial이기 때문에 One-Hot Encoding일 이용해 데이터 변환 필요
# # 0 => 1 0 0
# # 1 => 0 1 0
# # 2 => 0 0 1
# sess = tf.Session()
# t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
# t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

# # Placeholder
# X = tf.placeholder(shape=[None,2], dtype=tf.float32)
# T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# # Weight & bias
# W = tf.Variable(tf.random.normal([2,3]), name='weight')
# b = tf.Variable(tf.random.normal([3]), name='bias')

# # Hypothesis
# logit = tf.matmul(X,W) + b
# H = tf.nn.softmax(logit)

# # Loss Fucntion
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

# # Train
# train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# # 반복학습
# # parameter
# num_of_epoch = 1000
# batch_size = 100     # 한번에 학습할 x_data와 t_data의 행의 수

# def run_train(sess, train_x, train_t):
#     print('======= 학습시작 =======')
#     sess.run(tf.global_variables_initializer())
#     total_batch = int(train_x.shape[0] / batch_size)
    
#     for step in range(num_of_epoch):
#         for i in range(total_batch):
#             batch_x = train_x[i*batch_size:(i+1)*batch_size]
#             batch_t = train_t[i*batch_size:(i+1)*batch_size]
#             _, loss_val = sess.run([train,loss], feed_dict={X : batch_x, T : batch_t})

#         if step % 100 == 0 :
#             print('Loss : {}'.format(loss_val))
#     print('======= 학습종료 =======')
    
# # Evaluation(Accuracy)
# predict = tf.argmax(H,1)  # 1 => axis를 의미
# correct = tf.equal(predict, tf.argmax(T,1))
# accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# # Cross Validation
# cv = 5        # KFold의 K
# results = []   # 각 set에서 구한 accuracy를 집어넣을 list
# kf = KFold(n_splits=cv, shuffle=True)

# for training_idx, validation_idx in kf.split(x_data_train_norm):
#     train_x = x_data_train_norm[training_idx]
#     train_t = t_data_train_onehot[training_idx]
#     valid_x = x_data_train_norm[validation_idx]
#     valid_t = t_data_train_onehot[validation_idx]

#     run_train(sess, train_x, train_t)
#     results.append(sess.run(accuracy, feed_dict={X: valid_x, T : valid_t}))

# print('Cross Validation 결과 : {}'.format(np.mean(results)))

In [18]:
# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(train.iloc[:,1:], train['label'], test_size=0.3, random_state=0)

In [23]:
# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)  # scaling을 하기 위한 정보를 scaler에 setting

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

In [29]:
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))

In [34]:
# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([784,10]), name='weight')
b = tf.Variable(tf.random.normal([10]), name='bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)

# Loss Fucntion
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

# Train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# 반복학습
# parameter
num_of_epoch = 1000
batch_size = 100     # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess, train_x, train_t):
    print('======= 학습시작 =======')
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_x.shape[0] / batch_size)
    
    for step in range(num_of_epoch):
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X : batch_x, T : batch_t})

        if step % 100 == 0 :
            print('Loss : {}'.format(loss_val))
    print('======= 학습종료 =======')

In [35]:
# Evaluation(Accuracy)
predict = tf.argmax(H,1)  # 1 => axis를 의미
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# Cross Validation
cv = 5        # KFold의 K
results = []   # 각 set에서 구한 accuracy를 집어넣을 list
kf = KFold(n_splits=cv, shuffle=True)

for training_idx, validation_idx in kf.split(x_data_train_norm):
    train_x = x_data_train_norm[training_idx]
    train_t = t_data_train_onehot[training_idx]
    valid_x = x_data_train_norm[validation_idx]
    valid_t = t_data_train_onehot[validation_idx]

    run_train(sess, train_x, train_t)
    results.append(sess.run(accuracy, feed_dict={X: valid_x, T : valid_t}))

print('Cross Validation 결과 : {}'.format(np.mean(results)))

======= 학습시작 =======
Loss : 2.4283809661865234
Loss : 0.2629198133945465
Loss : 0.16819070279598236
Loss : 0.13415943086147308
Loss : 0.11908906698226929
Loss : 0.11135406792163849
Loss : 0.10683950781822205
Loss : 0.10393599420785904
Loss : 0.10193467885255814
Loss : 0.1004893034696579
======= 학습종료 =======
======= 학습시작 =======
Loss : 2.8176560401916504
Loss : 0.22051656246185303
Loss : 0.186706081032753
Loss : 0.1710643321275711
Loss : 0.16140617430210114
Loss : 0.1547219604253769
Loss : 0.1497219204902649
Loss : 0.14578860998153687
Loss : 0.14258462190628052
Loss : 0.13989873230457306
======= 학습종료 =======
======= 학습시작 =======
Loss : 1.9570811986923218
Loss : 0.2173765003681183
Loss : 0.18557173013687134
Loss : 0.16982045769691467
Loss : 0.1587662398815155
Loss : 0.1502857804298401
Loss : 0.1435655951499939
Loss : 0.13803061842918396
Loss : 0.1332874447107315
Loss : 0.12909863889217377
======= 학습종료 =======
======= 학습시작 =======
Loss : 2.003420352935791
Loss : 0.2533223628997803
Loss : 

In [36]:
# 학습진행
run_train(sess, x_data_train_norm, t_data_train_onehot)

# Accuracy 측정
result = sess.run(accuracy, feed_dict={X: x_data_test_norm, T : t_data_test_onehot})
print('최종 Accuracy : {}'.format(result))

# Prediction
test_scaled = scaler.transform(test)

result = sess.run(H, feed_dict={X : test_scaled})

======= 학습시작 =======
Loss : 1.5984625816345215
Loss : 0.21775935590267181
Loss : 0.17829474806785583
Loss : 0.16115707159042358
Loss : 0.15355142951011658
Loss : 0.14965103566646576
Loss : 0.14720752835273743
Loss : 0.1453894078731537
Loss : 0.14386913180351257
Loss : 0.1425083726644516
======= 학습종료 =======
최종 Accuracy : 0.9137301445007324
60


In [50]:
print(np.argmax(result[0]))
print(result.shape)

2
(28000, 10)


In [56]:
df = pd.DataFrame()
imageid = list(range(1,result.shape[0]+1))
label = []

for i in range(result.shape[0]):
    label.append(np.argmax(result[i]))
    
df['ImageId'] = imageid
df['Label'] = label

df.to_csv('./submission.csv', index=False)